In [17]:

import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

import funclib.pandaslib as pandaslib
import xlwings

import pandas as pd
import funclib.baselib as baselib
import funclib.iolib as iolib


class Game():
    '''game cls'''
    def __init__(self, url, full_title, score=0, votes=0, year_released=None):
        if not isinstance(full_title, str): raise ValueError('Full title was not a string')
        if full_title is None: raise ValueError('Full title was None')
        

        self.full_title = full_title.lower()
        self.year_released = year_released
        self.score = score
        self.votes = votes
        self.url = url
  
        
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def normalize(text):
    stem_t = text.lower().translate(remove_punctuation_map)
    return stem_t

vectorizer = TfidfVectorizer(tokenizer=normalize)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [23]:
games_root = 'C:/temp/c64/Games'
copy_games_to = 'C:/temp/c64_best/Games'

games = baselib.unpickle('c:/temp/c64games.pkl')
games_names = []
games_scores = []
games_votes = []
games_val = []
games_yr = []

exclude = ['german']



for g in games:
    skip = False
    if ((g.score > 7.5 and (g.score * g.votes > 200)) or (g.score * g.votes > 600)) and g.year_released < 1992:
        for s in exclude:
            if s in g.full_title:
                skip = True
                break
        if skip: continue
        games_names.append(g.full_title)
        games_scores.append(g.score)
        games_votes.append(g.votes)
        games_val.append(g.votes * g.score)
        games_yr.append(g.year_released)
    

                            
game_folders = []
game_folders_full = []

for folder, filename, extension, fullname in iolib.file_list_generator_dfe(games_root, '*.*', recurse=True):
    fld = folder.split('\\')[-1]
    if len(fld) == 1: continue
    if not fld in game_folders:
        game_folders.append(fld)
        game_folders_full.append(folder)

def get_game_sim(gname):
    lst = [cosine_sim(gname, fld) for fld in game_folders]
    return lst

print('%s good games, %s rom folders' % (len(games_names), len(game_folders)))

525 good games, 4844 rom folders


In [5]:
folders_full = []
folders = []

PP = iolib.PrintProgress(games_names)
for g in games_names:
    L = get_game_sim(g)
    folders_full.append(game_folders_full[L.index(max(L))])
    folders.append(game_folders[L.index(max(L))])
    PP.increment()
    #iolb.folder_copy
  

KeyboardInterrupt: 

In [55]:
df_good = pd.DataFrame(list(zip(games_names,games_yr, games_scores,games_votes,games_val, folders_full, folders)), columns=['name', 'yr', 'score','vote','val','folders_full','folders'])
#xlwings.view(df_good)

In [25]:
game_folders[121],game_folders_full[121]

('Acia', 'C:\\temp\\c64\\Games\\A\\Acia')

In [7]:
L


[0.21524464775514915,
 0.45688399134650726,
 0.3408242166238352,
 0.5739845431299192,
 0.27205959041240824,
 0.4627331204501844,
 0.4073526042885674,
 0.5029292651136352,
 0.23790309463326234,
 0.47507284504977726,
 0.275195812175023,
 0.272074731667965,
 0.3950018170571616,
 0.14976221961074723,
 0.14012086798868084,
 0.24235771956792357,
 0.47123047348238734,
 0.5898728381349829,
 0.24844640021645958,
 0.0845798608014702,
 0.2571550230878478,
 0.15962777427982133,
 0.20613696606828605,
 0.15592892548708362,
 0.3648274847169473,
 0.2482946073294489,
 0.26050181704867925,
 0.17707518167992103,
 0.27908738988811543,
 0.20892811860510968,
 0.0,
 0.14976221961074723,
 0.47432980456918206,
 0.44141890396079,
 0.4502681446556265,
 0.31025063093849464,
 0.39977332395721893,
 0.371515139140505,
 0.4334343289972558,
 0.38405571843526176,
 0.23757840886806497,
 0.0845798608014702,
 0.38405571843526176,
 0.40000863697542527,
 0.5187954242868258,
 0.3249317266808787,
 0.20186771310742205,
 0.4287

In [8]:
g

'draconus'

In [9]:
game_folders_full[L.index(max(L))]

'C:\\temp\\c64\\Games\\B\\Batman The Movie'

In [10]:
max(L)

0.9999999999999997

In [11]:
game_folders[L.index(max(L))]

'Doriath'

In [12]:
L.index(max(L))

1209

In [20]:
game_folders[0]

'Hole in One'

In [27]:
L = get_game_sim('action biker')
game_folders_full[L.index(max(L))], game_folders[L.index(max(L))]


('C:\\temp\\c64\\Games\\A\\Action Biker', 'Action Biker')